In [ ]:
import scanpy as sc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', 500)

import warnings
warnings.filterwarnings("ignore")

from matplotlib.lines import Line2D
from rpy2.robjects import pandas2ri
from rpy2.robjects import r
import rpy2.rinterface_lib.callbacks
import anndata2ri
import rpy2.robjects.numpy2ri
#import numpy2ri
import anndata
from matplotlib.ticker import MaxNLocator
pandas2ri.activate()
anndata2ri.activate()
rpy2.robjects.numpy2ri.activate()

plt.rcParams.update({
    'font.family': 'Arial'
})

# Function determining cutoff for cell positive for marker
from sklearn.mixture import GaussianMixture as GMM
def expression_cutoff(gene, adata_temp):
    data = adata_temp[: , gene].X.toarray()
    gmm = GMM(n_components=2).fit(data)
    labels = gmm.predict(data)
    df = pd.DataFrame({'expression': data.ravel(), 'label': labels})
    label0 = df[df['label'].isin([0])]['expression'].values
    label1 = df[df['label'].isin([1])]['expression'].values
    #print(label0.min())
    if label0.min() > label1.min():
        return label0.min()
    return label1.min()

%load_ext rpy2.ipython

In [ ]:
adata_early = sc.read('../Data/SC/parse_annotated_early_stage.h5ad')


# Early stage

## BArplot

In [ ]:
adata_early = sc.read('../Data/SC/parse_annotated_early_stage.h5ad')


In [ ]:
count_l, count_dict = [], {}
for sample in adata_early.obs.bmp_treatment.cat.categories.tolist():
    count_l = []    
    for cell_type in adata_early.obs.Cell_types.cat.categories.tolist():
        count_l.append(adata_early[adata_early.obs['Cell_types'] == cell_type].obs['bmp_treatment'].value_counts().reindex(adata_early.obs.bmp_treatment.cat.categories, fill_value=0)[sample])
    

    count_l = [i/sum(count_l) for i in count_l]
    count_dict[sample] = count_l

df = pd.DataFrame.from_dict(count_dict, orient='index', columns=adata_early.obs.Cell_types.cat.categories.tolist())
df.loc[-1] = adata_early.uns['Cell_types_colors']
df.index = ['color' if x==-1 else x for x in list(df.index)]

# Sort the DF
df = df[df.loc['no BMP'].sort_values(ascending=False).index]

df

In [ ]:

bmp_conditions = df.iloc[:-1].index.tolist()

# Create a subplot with 1 row and 4 columns
fig, axes = plt.subplots(1, 5, figsize=(14, 5), dpi=400)

for i, bmp in enumerate(bmp_conditions):
    df_flipped = df.iloc[:-1].loc[bmp].reset_index()
    df_flipped.columns = ['Cell_Type', 'Percentage']
    
    # Create the barplot on the respective axis
    sns.barplot(
        data=df_flipped, 
        y="Cell_Type",  # Cell types on the y-axis
        x="Percentage",  # Percentages on the x-axis
        palette=df.loc['color'].values,
        ax=axes[i]  # Specify the axis for each plot
    )
    
    # Set the title and other plot customizations
    axes[i].set_title(bmp, fontsize=22)
    axes[i].set_xlabel("")
    axes[i].set_ylabel("")
    
    cell_type_colors = dict(zip(df.columns, df.iloc[-1]))
    
    axes[i].tick_params(axis='x', labelsize=15)
    axes[i].tick_params(axis='y', labelsize=20)
    
    # Add a vertical color column next to the y-ticks in the first subplot
    if i == 0:  
        ytick_positions = axes[i].get_yticks()
        for y_pos, cell_type in zip(ytick_positions, df_flipped["Cell_Type"]):
            color = cell_type_colors.get(cell_type, "black")  # Get color or default to black
            axes[i].add_patch(plt.Rectangle((-0.055, y_pos - 0.4), 0.03, 0.8, color=color, transform=axes[i].transData, clip_on=False))
            axes[i].tick_params(axis='y', pad=11)
            axes[i].tick_params(axis='y', length=3.5, width=2)

    else:
        axes[i].tick_params(axis='y', which='both', length=0)  # Remove y-ticks
        axes[i].set_yticklabels([])  # Remove y-tick labels

    axes[i].set_xticks([0.2,0.4,0.6])
    axes[i].set_xlim(0,0.7)


# Adjust layout and show the plot

plt.tight_layout(pad=0.2)
plt.savefig('figures/d16_cluster_proportions.pdf')
plt.show()

In [ ]:
adata_early.obs["Cell_types"] = pd.Categorical(
    values=adata_early.obs.Cell_types, categories=cell_type_colors.keys(), ordered=True
)

with plt.rc_context({"figure.dpi":  300}): 

    sc.pl.umap(adata_early, color='Cell_types', palette = list(cell_type_colors.values()) ,frameon=False, size=10,save='_d16_legend.pdf' )

# DEG analysis


In [ ]:
%%R -i adata_early

Csparse_validate = "CsparseMatrix_validate"
library(Seurat)
library(edgeR)

seur <- as.Seurat(adata_early, counts = "counts", data = NULL)

#seur <- readRDS("Data/d50_d70_neurons_seurat.rds")
seur <- RenameAssays(seur, originalexp="RNA")

y <- Seurat2PB(seur, sample = "sample", cluster = "Cell_types")
keep.samples <- y$samples$lib.size > 5e4
y <- y[, keep.samples]
keep.genes <- filterByExpr(y, group=y$samples$cluster)
y <- y[keep.genes, , keep=FALSE]
y <- normLibSizes(y)

cluster <- as.factor(y$samples$cluster)

batch <- factor(y$samples$sample)
#design <- model.matrix(~ cluster + batch)
design <- model.matrix(~ cluster)

colnames(design) <- gsub("batch", "", colnames(design))
colnames(design)[1] <- "Int"
head(design)

y <- estimateDisp(y, design, robust=TRUE)
fit <- glmQLFit(y, design, robust=TRUE)

ncls <- nlevels(cluster)
contr <- rbind( matrix(1/(1-ncls), ncls, ncls), matrix(0, ncol(design)-ncls, ncls) )
diag(contr) <- 1
contr[1,] <- 0
rownames(contr) <- colnames(design)
colnames(contr) <- paste0("cluster", levels(cluster))
contr

qlf <- list()
for(i in 1:ncls){
 qlf[[i]] <- glmQLFTest(fit, contrast=contr[,i])
 qlf[[i]]$comparison <- paste0("cluster", levels(cluster)[i], "_vs_others")
}

top <- 500
topMarkers <- list()

de_df = data.frame(matrix( 
  vector(), 0, 7, dimnames=list(c(), c("gene","logFC","logCPM","F","PValue","FDR",'comparison'))), 
                stringsAsFactors=F)

for(i in 1:ncls) {
    #print(head(qlf[[i]])$comparison)
    ord <- order(qlf[[i]]$table$PValue, decreasing=FALSE)
    up <- qlf[[i]]$table$logFC[ord] > 0
    topMarkers[[i]] <- rownames(y)[ord[up][1:top]]
    #genes = 
    df = as.data.frame(topTags(qlf[[i]], n='all'))
    df =df[rownames(df) %in% rownames(y)[ord[up][1:top]], ]  
    df$comparison <- head(qlf[[i]])$comparison
    de_df = rbind(de_df, df)
}
print(dim(de_df))

write.csv(de_df, "parse_early_stage_bmp_deg.csv")

In [ ]:
de_genes = pd.read_csv('parse_early_stage_bmp_deg.csv',index_col=0)
de_genes['cluster'] = de_genes['comparison'].str.extract(r'cluster(.*?)_vs_others')
de_genes = de_genes[['gene','logFC','logCPM','F','PValue','FDR','comparison','cluster']]
de_genes = de_genes[(de_genes.FDR < 0.05) & (de_genes.logFC > 1)]
#de_genes = de_genes[~de_genes['gene'].str.startswith(('ENSG', 'LINC'))]
de_genes.to_excel("ARC_VMH_analysis/DE_lists/d16_bmp_de_list.xlsx")  

de_genes

# Late stage

In [ ]:
adata_late = sc.read('../Data/SC/parse_annotated_late_stage.h5ad')



In [ ]:
genes = ['SHH', 'POMC','ISL1','SIX6','PTCH1']

with plt.rc_context({ "figure.dpi": 300}):
    for gene in genes:
        sc.pl.umap(adata_late, color=gene, cmap='jet', frameon=False, size=10, colorbar_loc=None, save = f'_{gene}.pdf')

# Barplot

In [ ]:
count_l, count_dict = [], {}
for sample in adata_late.obs.bmp_treatment.cat.categories.tolist():
    count_l = []    
    for cell_type in adata_late.obs.Cell_types.cat.categories.tolist():
        count_l.append(adata_late[adata_late.obs['Cell_types'] == cell_type].obs['bmp_treatment'].value_counts().reindex(adata_late.obs.bmp_treatment.cat.categories, fill_value=0)[sample])
    

    count_l = [i/sum(count_l) for i in count_l]
    count_dict[sample] = count_l

df = pd.DataFrame.from_dict(count_dict, orient='index', columns=adata_late.obs.Cell_types.cat.categories.tolist())
df.loc[-1] = adata_late.uns['Cell_types_colors']
df.index = ['color' if x==-1 else x for x in list(df.index)]

# Sort the DF
df = df[[ 'Telencephalic neurons','Unassigned','Astrocytes','Optic area neurons', 'LHX1+/ARX+',
       'OTP+/SST+/BNC2+','GPR149+/LHX8+', 'POMC+/SOX14+/NR5A1+', 'FEZF1+/SOX14+', 
       'NR5A2+/ONECUT1/3+', 'DLX6-AS1+/FOXP2+',   'GHRH+/PNOC+',
       'Immature ARC neurons', 'Tanycytes', 'POMC+/TBX3+/NR5A2+','AGRP+/OTP+','PNOC+/NPFFR2+']]



df


In [ ]:


bmp_conditions = df.iloc[:-1].index.tolist()

# Create a subplot with 1 row and 4 columns
fig, axes = plt.subplots(1, 4, figsize=(14, 5), dpi=400)

for i, bmp in enumerate(bmp_conditions):
    df_flipped = df.iloc[:-1].loc[bmp].reset_index()
    df_flipped.columns = ['Cell_Type', 'Percentage']
    
    # Create the barplot on the respective axis
    sns.barplot(
        data=df_flipped, 
        y="Cell_Type",  # Cell types on the y-axis
        x="Percentage",  # Percentages on the x-axis
        palette=df.loc['color'].values,
        ax=axes[i]  # Specify the axis for each plot
    )
    
    # Set the title and other plot customizations
    axes[i].set_title(bmp, fontsize=22)
    axes[i].set_xlabel("")
    axes[i].set_ylabel("")
    
    cell_type_colors = dict(zip(df.columns, df.iloc[-1]))
    
    axes[i].tick_params(axis='x', labelsize=15)
    axes[i].tick_params(axis='y', labelsize=20)
    
    # Add a vertical color column next to the y-ticks in the first subplot
    #if i == 0:  


    if i != 0:
        axes[i].tick_params(axis='y', which='both', length=0)  # Remove y-ticks
        axes[i].set_yticklabels([])  # Remove y-tick labels

    axes[i].set_xticks([0.2,0.4])
    axes[i].set_xlim(0,0.5)


# Adjust layout and show the plot

plt.tight_layout(pad=0.3)
plt.savefig('figures/d50_cluster_proportions.pdf')
plt.show()

In [ ]:


cell_types_order = [ 'Telencephalic neurons','Unassigned','Astrocytes','Optic area neurons', 'LHX1+/ARX+',
       'OTP+/SST+/BNC2+','GPR149+/LHX8+', 'POMC+/SOX14+/NR5A1+', 'FEZF1+/SOX14+', 
       'NR5A2+/ONECUT1/3+', 'DLX6-AS1+/FOXP2+',   'GHRH+/PNOC+',
       'Immature ARC neurons', 'Tanycytes', 'POMC+/TBX3+/NR5A2+','AGRP+/OTP+','PNOC+/NPFFR2+']
colors = adata_late.uns['Cell_types_colors']
print(len(cell_types_order))
print(len(colors))
colors_dict = dict(zip(list(adata_late.obs.Cell_types.cat.categories), colors))
colors_dict = {k: colors_dict[k] for k in cell_types_order if k in colors_dict}
colors_dict

adata_late.obs["Cell_types"] = pd.Categorical(
    values=adata_late.obs.Cell_types, categories=[ 'Telencephalic neurons','Unassigned','Astrocytes','Optic area neurons', 'LHX1+/ARX+',
       'OTP+/SST+/BNC2+','GPR149+/LHX8+', 'POMC+/SOX14+/NR5A1+', 'FEZF1+/SOX14+', 
       'NR5A2+/ONECUT1/3+', 'DLX6-AS1+/FOXP2+',   'GHRH+/PNOC+',
       'Immature ARC neurons', 'Tanycytes', 'POMC+/TBX3+/NR5A2+','AGRP+/OTP+','PNOC+/NPFFR2+'], ordered=True
)

In [ ]:
with plt.rc_context({"figure.dpi":  300}): 

    sc.pl.umap(adata_late, color='Cell_types', palette = list(colors_dict.values()) ,frameon=False, size=10,save='_legend.pdf' )

# DEG

In [ ]:
%%R -i adata_late

Csparse_validate = "CsparseMatrix_validate"
library(Seurat)
library(edgeR)

seur <- as.Seurat(adata_late, counts = "counts", data = NULL)

#seur <- readRDS("Data/d50_d70_neurons_seurat.rds")
seur <- RenameAssays(seur, originalexp="RNA")

y <- Seurat2PB(seur, sample = "sample", cluster = "Cell_types")
keep.samples <- y$samples$lib.size > 5e4
y <- y[, keep.samples]
keep.genes <- filterByExpr(y, group=y$samples$cluster)
y <- y[keep.genes, , keep=FALSE]
y <- normLibSizes(y)

cluster <- as.factor(y$samples$cluster)

batch <- factor(y$samples$sample)
#design <- model.matrix(~ cluster + batch)
design <- model.matrix(~ cluster)

colnames(design) <- gsub("batch", "", colnames(design))
colnames(design)[1] <- "Int"
head(design)

y <- estimateDisp(y, design, robust=TRUE)
fit <- glmQLFit(y, design, robust=TRUE)

ncls <- nlevels(cluster)
contr <- rbind( matrix(1/(1-ncls), ncls, ncls), matrix(0, ncol(design)-ncls, ncls) )
diag(contr) <- 1
contr[1,] <- 0
rownames(contr) <- colnames(design)
colnames(contr) <- paste0("cluster", levels(cluster))
contr

qlf <- list()
for(i in 1:ncls){
 qlf[[i]] <- glmQLFTest(fit, contrast=contr[,i])
 qlf[[i]]$comparison <- paste0("cluster", levels(cluster)[i], "_vs_others")
}

top <- 500
topMarkers <- list()

de_df = data.frame(matrix( 
  vector(), 0, 7, dimnames=list(c(), c("gene","logFC","logCPM","F","PValue","FDR",'comparison'))), 
                stringsAsFactors=F)

for(i in 1:ncls) {
    #print(head(qlf[[i]])$comparison)
    ord <- order(qlf[[i]]$table$PValue, decreasing=FALSE)
    up <- qlf[[i]]$table$logFC[ord] > 0
    topMarkers[[i]] <- rownames(y)[ord[up][1:top]]
    #genes = 
    df = as.data.frame(topTags(qlf[[i]], n='all'))
    df =df[rownames(df) %in% rownames(y)[ord[up][1:top]], ]  
    df$comparison <- head(qlf[[i]])$comparison
    de_df = rbind(de_df, df)
}
print(dim(de_df))

write.csv(de_df, "parse_late_stage_bmp_deg.csv")

In [ ]:
de_genes = pd.read_csv('parse_late_stage_bmp_deg.csv',index_col=0)
de_genes['cluster'] = de_genes['comparison'].str.extract(r'cluster(.*?)_vs_others')
de_genes = de_genes[['gene','logFC','logCPM','F','PValue','FDR','comparison','cluster']]
de_genes = de_genes[(de_genes.FDR < 0.05) & (de_genes.logFC > 1)]
#de_genes = de_genes[~de_genes['gene'].str.startswith(('ENSG', 'LINC'))]
de_genes.to_excel("ARC_VMH_analysis/DE_lists/d50+_bmp_de_list.xlsx")  

de_genes



# POMC subtype DE analysis

In [ ]:
adata_pomc = adata_late[adata_late.obs.Cell_types.isin(['POMC+/SOX14+/NR5A1+', 'POMC+/TBX3+/NR5A2+'])]


In [ ]:
%%R -i adata_pomc

Csparse_validate = "CsparseMatrix_validate" 
library(Seurat)
library(edgeR)

seur <- as.Seurat(adata_pomc, counts = "counts", data = NULL)

#seur <- readRDS("Data/d50_d70_neurons_seurat.rds")
seur <- RenameAssays(seur, originalexp="RNA")

y <- Seurat2PB(seur, sample = "batch", cluster = "Cell_types")
keep.samples <- y$samples$lib.size > 5e4
y <- y[, keep.samples]
keep.genes <- filterByExpr(y, group=y$samples$cluster)
y <- y[keep.genes, , keep=FALSE]
y <- normLibSizes(y)

print('OK')
cluster <- as.factor(y$samples$cluster)

batch <- factor(y$samples$sample)
design <- model.matrix(~ cluster + batch)
colnames(design) <- gsub("batch", "", colnames(design))
colnames(design)[1] <- "Int"
head(design)

y <- estimateDisp(y, design, robust=TRUE)
fit <- glmQLFit(y, design, robust=TRUE)
print('OK')

ncls <- nlevels(cluster)
contr <- rbind( matrix(1/(1-ncls), ncls, ncls), matrix(0, ncol(design)-ncls, ncls) )
diag(contr) <- 1
contr[1,] <- 0
rownames(contr) <- colnames(design)
colnames(contr) <- paste0("cluster", levels(cluster))
contr

qlf <- list()
for(i in 1:ncls){
 qlf[[i]] <- glmQLFTest(fit, contrast=contr[,i])
 qlf[[i]]$comparison <- paste0("cluster", levels(cluster)[i], "_vs_others")
}
print('OK')

top <- 20000
topMarkers <- list()

de_df = data.frame(matrix( 
  vector(), 0, 7, dimnames=list(c(), c("gene","logFC","logCPM","F","PValue","FDR",'comparison'))), 
                stringsAsFactors=F)

for(i in 1:ncls) {
    #print(head(qlf[[i]])$comparison)
    ord <- order(qlf[[i]]$table$PValue, decreasing=FALSE)
    up <- qlf[[i]]$table$logFC[ord] > 0
    topMarkers[[i]] <- rownames(y)[ord[up][1:top]]
    #genes = 
    df = as.data.frame(topTags(qlf[[i]], n='all'))
    df =df[rownames(df) %in% rownames(y)[ord[up][1:top]], ]  
    df$comparison <- head(qlf[[i]])$comparison
    de_df = rbind(de_df, df)
}
print(dim(de_df))

write.csv(de_df, "parse_pomc.csv")

In [ ]:
%%R -o markers
library(Seurat)
# Identify spatially variable genes in spatial human hypomap
merged <- readRDS('../Data/SC/humanHYPOMAP_spatial.rds')
merged_subset = subset(x = merged, subset = regional_clusters_named %in% c('ARC',"VMH"))
Idents(object = merged_subset) <- "regional_clusters_grouped"
markers <- FindAllMarkers(object = merged_subset, features = rownames(merged_subset))

In [ ]:
de_genes = pd.read_csv('parse_pomc.csv',index_col=0)
de_genes['cluster'] = de_genes['comparison'].str.extract(r'cluster(.*?)_vs_others')
de_genes = de_genes[['gene','logFC','logCPM','F','PValue','FDR','comparison','cluster']]
de_genes['identity'] = de_genes.cluster.map({'POMC+/TBX3+/NR5A2+': 'ARC','POMC+/SOX14+/NR5A1+':'VMH'})


arc_genes = list(markers[(markers.cluster == 'ARC') & (markers.p_val_adj < 0.05) & (markers.avg_log2FC >0.1)].gene.values)
vmh_genes = list(markers[(markers.cluster == 'VMH') & (markers.p_val_adj < 0.05) & (markers.avg_log2FC >0.1)].gene.values)

de_genes['spatial_hypomap_de_gene'] = '-'
de_genes.loc[ (de_genes.gene.isin(arc_genes)), 'spatial_hypomap_de_gene'] =  'ARC'
de_genes.loc[ (de_genes.gene.isin(vmh_genes)), 'spatial_hypomap_de_gene'] =  'VMH'

top_genes = de_genes[(de_genes.FDR < 0.05) & (de_genes.logFC > 1)]
top_genes.to_excel('arc_vmh_pomc_subtype_deg.xlsx')


top_genes = (top_genes.loc[top_genes['spatial_hypomap_de_gene'].isin(['VMH','ARC'])].sort_values(by=['cluster', 'logFC'], ascending=[True, False]))

de_genes.loc[de_genes.cluster == 'POMC+/SOX14+/NR5A1+','logFC'] = de_genes[de_genes.cluster == 'POMC+/SOX14+/NR5A1+']['logFC'] * -1



In [ ]:

from adjustText import adjust_text
with plt.rc_context({ "figure.dpi": 300, "figure.figsize": [6, 4 ] }):


    # highlight down- or up- regulated genes
    down = de_genes[(de_genes['logFC']<=-1)&(de_genes['FDR']<=0.05)]
    up = de_genes[(de_genes['logFC']>=1)&(de_genes['FDR']<=0.05)]
    plt.axvline(-1,color="grey",linestyle="--")
    plt.axvline(1,color="grey",linestyle="--")
    plt.axhline(-np.log10(0.05),color="grey",linestyle="--")
    plt.scatter(x=de_genes['logFC'],y=de_genes['FDR'].apply(lambda x:-np.log10(x)),s=4,label="Not significant",color="#bebebe")
    plt.scatter(x=down['logFC'],y=down['FDR'].apply(lambda x:-np.log10(x)),s=4,label='POMC+/SOX14+/NR5A1+',color="#a11d02")
    
    plt.scatter(x=up['logFC'],y=up['FDR'].apply(lambda x:-np.log10(x)),s=4,label='POMC+/TBX3+/NR5A2+',color="#eddb7e")

    highlight_df_vmh = de_genes[de_genes['gene'].isin(top_genes[top_genes.spatial_hypomap_de_gene == 'VMH'].gene)]
    highlight_df_arc = de_genes[de_genes['gene'].isin(top_genes[top_genes.spatial_hypomap_de_gene == 'ARC'].gene)]
    

    # Circle the selected genes (larger, transparent points)
    plt.scatter(highlight_df_vmh['logFC'], -np.log10(highlight_df_vmh['FDR']), s=6.5, facecolors='none', edgecolors='orange', linewidths=0.7, label="Spatial human VMH DEG")
    plt.scatter(highlight_df_arc['logFC'], -np.log10(highlight_df_arc['FDR']), s=6.5, facecolors='none', edgecolors='black', linewidths=0.7, label="Spatial human ARC DEG")
    
    gene_list  =['SOX14','GPR149','CBLN1','NR5A1','GABRB2','SOX1','CDH13','CA10','CNR1','GABRA1','HS3ST4','ST6GAL2','GABRE',
             'IL13RA1','NR5A2','IL13RA2','RADX','ECEL1','LHX2','GBE1','AK7','ANXA2','PROX1','NR3C1']
    texts=[]
    for gene in gene_list:
        texts.append(plt.text(x=de_genes[de_genes.gene == gene]['logFC'],y=-np.log10(de_genes[de_genes.gene == gene]['FDR']),s=gene,fontsize=8,weight='bold'))
    adjust_text(texts, expand=(1.65, 1.7),arrowprops=dict(arrowstyle="-", color='black', lw=0.8, shrinkA=2, shrinkB=2))
    
    plt.ylim(0,7.8)
    plt.xlim(-11.5,7.5)
    
    plt.xlabel("logFC", size=14)
    plt.ylabel("-log10(p-value)", size=14)
    
    plt.title('POMC subtype DEG', fontsize=16)
    
    legend_handles = [
        Line2D([0], [0], marker='o', color='w', markerfacecolor='#bebebe', markersize=5, label="Not significant"),
        Line2D([0], [0], marker='o', color='w', markerfacecolor='#a11d02', markersize=5, label='POMC+/SOX14+/NR5A1+'),
        Line2D([0], [0], marker='o', color='w', markerfacecolor='#eddb7e', markersize=5, label='POMC+/TBX3+/NR5A2+'),
        Line2D([0], [0], marker='o', color='w', markerfacecolor='none', markeredgecolor='#F58024', markersize=4, markeredgewidth=2.5, label="Spatial hypomap VMH DEG"),
        Line2D([0], [0], marker='o', color='w', markerfacecolor='none', markeredgecolor='black', markersize=4, markeredgewidth=2.5, label="Spatial hypomap ARC DEG")
    ]

    # Add legend with increased linewidth
    plt.legend(handles=legend_handles, loc="upper left", bbox_to_anchor=(0.98, 0.75), 
               fontsize=10, frameon=False, markerscale=3, handletextpad=0.05, handleheight=2, handlelength=2)


    plt.tight_layout()
    plt.savefig('figures/POMC_DE_analysis.pdf')
    plt.show()